In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

In [2]:
class OneToOneITU():
    def __init__(self, n, m, parameters = (None,None) , lbs=(0, 0)):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
     
       
        self.A_ij = parameters[0]
        self.B_ij = parameters[1]

    def D_ij(self, U_i, V_j):
        return (U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij)/(1+ self.A_ij)

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

In [3]:
# def transpose_mkt(self):
#     n, m = self.n, self.m
#     self.m = n
#     self.n = m

#     self.A_ij = 1 / self.A_ij.T
#     self.B_ij = self.B_ij.T/ self.A_ij

# OneToOneITU.transpose_mkt = transpose_mkt

In [4]:
def check_primal_feas(self, mu_ij):
    print(f"i: { np.all(np.sum(mu_ij,axis=1) <= np.ones(self.n))}, j:  {np.all(np.sum(mu_ij,axis=0) <= np.ones(self.m)) }")
    print(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}")

OneToOneITU.check_primal_feas = check_primal_feas

def check_IR(self,mu_ij,U_i,V_j, output = False):
    IR_i = np.sum((U_i - self.lb_U)* (1- np.sum(mu_ij,axis=1)))
    IR_j = np.sum((V_j - self.lb_V) * (1 - np.sum(mu_ij,axis=0))) 
    print(f"i: { IR_i}, j:  {IR_j }")
    if output is True:
        IR_i = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        IR_j = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return IR_i, IR_j
    
OneToOneITU.check_IR = check_IR

def check_CS(self,U_i,V_j, output = None):
    CS = np.minimum(self.D_ij(U_i,V_j),0)
    if output:
        return CS, np.where(CS < 0 )
    print(f"min_ij D_ij : {(self.D_ij(U_i,V_j)).min()} ")
    
OneToOneITU.check_CS = check_CS 

def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    print("FEAS")
    self.check_primal_feas(mu_ij)
    print("________________")
    print("ε-CS")
    self.check_CS(U_i, V_j )
    print("________________")
    print("IR")
    self.check_IR(mu_ij, U_i, V_j )

OneToOneITU.check_all = check_all

# Auction algorithms

### Forward Auction

In [5]:
def forward_auction(self, data = None, tol_ε = None):
    if data is None:
        V_j = np.ones(self.m) * self.lb_V
        mu_ij = np.ones( self.n, dtype= int) * (self.m +1)
    else:
        mu_ij_01 = data[0]
        V_j = data[1]
        id_i, id_j = np.where(mu_ij_01[:,:-1] >0)
        mu_ij = np.ones(self.n, dtype= int) * (self.m+1)
        mu_ij[id_i] = id_j

    unassigned = np.where(mu_ij == self.m + 1)[0] 
    n_unassigned = len(unassigned)
    iter = 0

    while n_unassigned > 0:
        iter += 1
        U_ij = self.get_U_ij( V_j, unassigned)
        perm_unmatched = np.all(U_ij <= self.lb_U, axis = 1)
        
        if np.any(perm_unmatched):
            mu_ij[unassigned] += (self.m - mu_ij[unassigned]) * perm_unmatched
        
        else:
            ### bidding phase
            U_ij = np.concatenate((U_ij, self.lb_U * np.ones((n_unassigned,1))), axis = 1)
            j_i = np.argmax(U_ij, axis= 1)
            masked_U_ij = np.where(j_i[:,None] == np.arange(self.m+1)[None, :], -np.inf,U_ij)
            w_i = np.max(masked_U_ij, axis=1)
            j_i_unique = np.unique(j_i)
            offers = np.where(j_i[:,None] == j_i_unique[None,:], 
                              self.get_V_ij(unassigned[:,None],j_i_unique[None,:],w_i[:,None]), 
                              np.nan)

            ### assignment phase
            i_j_among_unass = np.nanargmax(offers, axis = 0)
            i_j = unassigned[ i_j_among_unass]
            best_offer_j = offers[i_j_among_unass, np.arange(len(j_i_unique))]

            # modify solution
            mu_ij[  np.any( mu_ij[:,None] == j_i_unique , axis = 1) ] = self.m +1
            mu_ij[i_j] = j_i_unique
            V_j[j_i_unique] += np.maximum(best_offer_j - V_j[j_i_unique], tol_ε)
        
        unassigned = np.where(mu_ij == self.m + 1)[0]
        n_unassigned = len(unassigned)
        
    print(f"for{iter}")
    matched_i =  mu_ij < self.m
    U_i = np.ones(self.n) * self.lb_U
    U_i[matched_i] = self.get_U_ij(V_j, matched_i)[np.arange(matched_i.sum()), mu_ij[matched_i]]
    mu_ij_01 = mu_ij[:,None] == np.arange(self.m +1)

    return mu_ij_01, U_i, V_j
    

OneToOneITU.forward_auction = forward_auction

#### Example forward auction

In [6]:
# n_i = 400
# m_j = 300
# np.random.seed(430)
# A_ij = (np.random.normal(2,2,[n_i,m_j]).round(0))**2+1
# B_ij = np.random.normal(40,5,[n_i,m_j]).round(0)

# example_mkt = OneToOneITU( n_i,m_j, (A_ij,B_ij), tol=None)
# mu_ij_01, u_i, v_j = example_mkt.forward_auction(tol_ε = .0001)

In [7]:
# example_mkt.forward_auction((mu_ij_01, v_j ), tol_ε = 5)

In [8]:
# example_mkt.check_all((mu_ij_01[:,:-1], u_i, v_j))

#### TU

In [9]:
# n_TU = 25
# m_TU = 20
# A_TU = np.ones([n_TU,m_TU])
# B_TU = np.random.normal(40,5,[n_TU,m_TU]).round(0)

In [10]:
# example_mkt = OneToOneITU( n_TU,m_TU, (A_TU,B_TU), tol=0.000001)
# mu_TU, u_TU, v_TU = example_mkt.forward_auction()

In [11]:
# example_mkt = OneToOneITU( n_TU,m_TU, (A_TU,B_TU), tol=0.000001)
# mu_TU, u_TU, v_TU = example_mkt.forward_auction()
# def solve_1to1(Φ_i_j):
#     n, m = np.shape(Φ_i_j)
#     M_z_a = spr.bmat([[spr.kron(spr.eye(n), np.ones((1, m)))],
#                       [spr.kron(np.ones((1, n)), spr.eye(m))]])
  
#     q = np.concatenate((np.ones(n), np.ones(m)))

#     model = grb.Model()
#     mu_a = model.addMVar(n * m, vtype=grb.GRB.INTEGER)
#     model.setObjective(Φ_i_j.flatten() @ mu_a, GRB.MAXIMIZE)
#     model.addConstr(M_z_a @ mu_a <= q)
    
#     model.Params.OutputFlag = 0
#     model.optimize()

#     return np.array(model.x, dtype= bool).reshape([n,m])*1

# mu_gurobi = solve_1to1(B_TU)
# print((mu_gurobi * B_TU).sum())
# print(u_TU.sum() +v_TU.sum())

### Backward Auction

In [12]:
def backward_auction(self, data, tol_ε, max_back_iter = 30):
    mu_ij_01, U_i, V_j = data
    id_i, id_j = np.where(mu_ij_01[:,:-1] >0)
    mu_ij = np.ones(self.m) * self.n
    mu_ij[id_j] = id_i

    cont = True
    iter = 1
    while cont and iter < max_back_iter:
        iter += 1
     
        ### bidding phase
        V_ij = np.concatenate((self.get_V_ij(np.arange(self.n)[:,None], np.ones(self.m, dtype= bool),U_i[:,None] + tol_ε),
                                np.ones(self.m)[None,:]* self.lb_V), axis = 0 )
        i_j = np.argmax(V_ij, axis=0)
        masked_V = np.where(np.arange(self.n+1)[:,None] == i_j[None, :], -np.inf,V_ij)
        β_j = V_ij[i_j,np.arange(self.m)]
        V_j[((mu_ij == self.n) * (V_j - self.lb_V)> 0)] = self.lb_V
        γ_j = np.max(masked_V, axis=0)
      
        j_bidding = β_j > V_j
        if np.any (j_bidding):
            i_j_bidding = i_j[None,j_bidding]
            i_j_unique = np.unique(i_j_bidding)
            bids_ij = self.get_U_ij(γ_j,i_j_unique[:,None], j_bidding) 
            bids_ij = np.where(i_j_unique[:,None] == i_j_bidding[None,:], bids_ij, np.nan)[0]

            ### assignment phase
            j_i = np.nanargmax(bids_ij, axis=1)
            max_bids_i = bids_ij[range(len(i_j_unique)),j_i]
           
            # return to original indeces
            j_i = np.where(j_bidding >0)[0][j_i]
            i_j_unique = i_j[j_i]
            # modify mu,u,v
            U_i[i_j_unique] = max_bids_i
            V_j[j_i] = γ_j[j_i]
            mu_ij[  np.any( mu_ij[:,None] == i_j_unique , axis = 1) ] = self.n 
            mu_ij[j_i] = i_j_unique 
        else:
            cont = False
    print(iter)
    ####
    #print( np.any(((mu_ij == self.n) * (V_j - self.lb_V)> 0)))
    V_j[((mu_ij == self.n) * (V_j - self.lb_V)> 0)] = self.lb_V
    ####
    mu_ij_01 = np.concatenate((np.arange(self.n)[:,None] == mu_ij,
                                np.zeros(self.n)[:,None]), axis = 1)
    mu_ij_01[:,-1] = np.all(mu_ij_01 == 0, axis= 1)
    return mu_ij_01, U_i, V_j

OneToOneITU.backward_auction = backward_auction

#### Notes backward

In [13]:
# eps = 10

In [14]:
# mu_ij_01, u_i, v_j = example_mkt.forward_auction(tol_ε = eps)

In [15]:
# eps = eps /2

In [16]:
# mu_ij_01, u_i, v_j = example_mkt.backward_auction((mu_ij_01, u_i, v_j), eps)

In [17]:
# mu_ij_01, u_i, v_j = example_mkt.forward_auction((mu_ij_01, v_j ), tol_ε = eps)

In [18]:
# mu_ij_01, u_i, v_j = example_mkt.forward_auction( (mu_ij_01, v_j), .00001)

In [19]:
# for k in range(20):
#     eps = eps /2
#     mu_ij_01, u_i, v_j = example_mkt.backward_auction((mu_ij_01, u_i, v_j), eps)
#     mu_ij_01, u_i, v_j = example_mkt.forward_auction((mu_ij_01, v_j ), tol_ε = eps)



In [20]:
# example_mkt.check_all((mu_ij_01[:,:-1], u_i, v_j))

### Forward-backward auction

In [21]:
def forward_backward_scaling(self, tol_ε ,tol_ε_obj, scaling_factor, max_back_iter  ):
    mu_ij_01, u_i, v_j = self.forward_auction(tol_ε = tol_ε)
    # for k in range(10):
    while tol_ε > tol_ε_obj:
        tol_ε *=  scaling_factor
        # print("####")
        # print( tol_ε)
        mu_ij_01, u_i, v_j = self.backward_auction((mu_ij_01, u_i, v_j), tol_ε,max_back_iter)
        #print("done back")
        mu_ij_01, u_i, v_j = self.forward_auction((mu_ij_01, v_j ), tol_ε = tol_ε)
        #print(np.all((1-mu_ij_01[:,:-1].sum(1))*(u_i <= self.lb_U  ) ))
        #print("done for")
        #self.check_IR(mu_ij_01[:,:-1],u_i,v_j)
        #print(np.min(self.check_CS(u_i,v_j,True)[0]))
        if np.all(self.check_CS(u_i,v_j,True)[0]>=0):
            return mu_ij_01, u_i, v_j

    return mu_ij_01, u_i, v_j
OneToOneITU.forward_backward_scaling =forward_backward_scaling  

In [22]:
# n_i = 600
# m_j = 300
# np.random.seed(430)
# A_ij = (np.random.normal(2,2,[n_i,m_j]).round(0))**2+1
# B_ij = np.random.normal(40,5,[n_i,m_j]).round(0)

# example_mkt = OneToOneITU( n_i,m_j, (A_ij,B_ij))

In [23]:
# mu_ij_01_scaling, u_i_scaling, v_j_scaling = example_mkt.forward_backward_scaling(1,1e-7,scaling_factor= 1/2, max_back_iter = np.inf )

In [24]:
# example_mkt.check_all((mu_ij_01_scaling[:,:-1], u_i_scaling, v_j_scaling ))

In [25]:
# mu_ij_01_noscaling, u_i_noscaling, v_j_noscaling = example_mkt.forward_auction(tol_ε= 0.000001)

#### Another example

In [35]:
def generate_example(self):
    np.random.seed(1065470)
    k = 2

    θ_true = np.array([.4,-.2, 1,2])

    X_i = np.random.normal([5,1],[5,1], size = [self.n, k]).round(0)
    Y_j = np.random.normal([5,1],[5,1], size = [self.m, k]).round(0)
    #X_ijk = np.random.choice(10, size = (self.n_i,m_j, k))

    A = np.exp( np.abs((X_i[:,None,:]- Y_j[None,:,:]))@  θ_true[:k])
    B = np.abs((X_i[:,None,:]- Y_j[None,:,:])) @ θ_true[k:2*k] 
    print(len(np.unique(X_i)))
    self.A_ij = A
    self.B_ij = B

OneToOneITU.generate_example = generate_example

In [43]:
n_i = 300
m_j  = 309
ex2_mrk = OneToOneITU( n_i,m_j)
ex2_mrk.generate_example()

28


In [37]:
ex2_mrk.B_ij.min()

0.0

In [42]:
mu_ij_01_noscaling, u_i_noscaling, v_j_noscaling = ex2_mrk.forward_auction(tol_ε= .00001)

KeyboardInterrupt: 

In [44]:
mu_ij_01_scaling, u_i_scaling, v_j_scaling = ex2_mrk.forward_backward_scaling(1,1e-8,scaling_factor= 1/2, max_back_iter = np.inf )

for106
1247
for0
59
for50
1334
for0
59
for265
775
for155
1080
for105
1337
for218
1454
for311
1247
for384
2500
for798
6496
for162
5125
for180
5602
for192
5475
for185
4500
for199
6170
for238
5381
for216
4591
for117
1105
for770
947
for3
692
for3
859
for4
664
for3
1028
for4
959
for5
1192
for3
860
for3


In [45]:
ex2_mrk.check_all( (mu_ij_01_scaling[:,:-1], u_i_scaling, v_j_scaling ))

FEAS
i: True, j:  True
#matched: 300 over 300
________________
ε-CS
min_ij D_ij : -7.440458833048379e-09 
________________
IR
i: 0.0, j:  0.0


Next steps:
* $ij$-specific scaling
* max iter in the backward iteration
* (IMPORTANT) now all you are computing V_ij for all i and j in backward